# DAX: Ключевые функции Power BI
**CALCULATE + контекст = сила DAX**

## Содержание
1. Переменные и обработка ошибок  
2. Фильтры и контекст  
3. Агрегация и расчёты  
4. Условные расчёты  
5. Self-Join и сложные связи  
6. Иерархия дат — лучшие практики  
7. Функции дат и временные расчёты  
8. Агрегация и ранжирование  
9. Мера vs Вычисляемый столбец  
10. Производительность  
11. Шаблон сложной меры  

---

## 1. Переменные и обработка ошибок

```dax
Продажи с дисконтом =
VAR TotalSales = SUM(Sales[Amount])
VAR Discount = 0.1
VAR DiscountedSales = TotalSales * (1 - Discount)
RETURN
IF(DiscountedSales > 1000, DiscountedSales, TotalSales)
```

> **VAR** — однократное вычисление, отладка, читаемость

```dax
Безопасное деление = DIVIDE([Продажи], [Количество], 0)
Проверка на пустоту = IF(ISBLANK([Мера]), "Нет данных", [Мера])
```

> **DIVIDE** — всегда вместо `/`

## 2. Фильтры и контекст

```dax
Продажи Москва =
CALCULATE(
    SUM(Sales[Amount]),
    Customers[City] = "Москва"
)
```

> **CALCULATE** — меняет контекст фильтрации  
> **ALL()** — снимает фильтр: `CALCULATE(SUM(...), ALL(Date))` — не ставим, если нужна интерактивность  
> **FILTER()** — сложные условия: `CALCULATE(SUM(...), FILTER(Table, Condition))`

## 3. Агрегация и расчёты

```dax
% от общего =
DIVIDE(
    [Продажи],
    CALCULATE([Продажи], ALL(Sales))
)
```

> **DIVIDE** — безопасное деление  
> **ALL(Sales)** — игнорирует фильтры по строкам Sales

## 4. Условные расчёты

```dax
Категория =
SWITCH(
    TRUE(),
    [Продажи] > 10000, "High",
    [Продажи] > 1000,  "Medium",
    "Low"
)
```

> **SWITCH(TRUE(), ...)** — аналог CASE WHEN в SQL

## 5. Self-Join и сложные связи

```dax
Имя руководителя =
LOOKUPVALUE(
    Employees[Name],
    Employees[ID], Employees[ManagerID]
)
```

> **LOOKUPVALUE** — как self-join в SQL

## 6. Иерархия дат — лучшие практики

```dax
Calendar =
VAR Base = CALENDARAUTO()
RETURN
ADDCOLUMNS(
    Base,
    "Year", YEAR([Date]),
    "Quarter", "Q" & FORMAT([Date], "q"),
    "MonthName", FORMAT([Date], "MMMM", "ru-RU"),
    "MonthNum", MONTH([Date]),
    "MonthYear", FORMAT([Date], "MMM YYYY", "ru-RU"),
    "Week", "Неделя " & WEEKNUM([Date]),
    "Weekday", FORMAT([Date], "dddd", "ru-RU"),
    "Day", DAY([Date])
)
```

**Настройка в Power BI:**
1. **Связь:** `Calendar[Date]` → `Sales[OrderDate]` (1:many)  
2. **Mark as Date Table:** `Modeling → Mark as Date Table → Date`  
3. **Иерархия:** `Year → Quarter → MonthName → Day`  
4. **Сортировка:** `MonthName → Sort by Column → MonthNum`  
5. **Отключить авто-даты:** `File → Options → Data Load → Disable Auto Date/Time`

## 7. Функции дат и временные расчёты

```dax
-- Базовая мера
Продажи = SUM(Sales[Amount])

-- YoY
Продажи прошлый год = CALCULATE([Продажи], SAMEPERIODLASTYEAR('Calendar'[Date]))

YoY % = DIVIDE([Продажи] - [Продажи прошлый год], [Продажи прошлый год])

-- Умный YoY (скрыть на уровне дня)
YoY % (умный) = IF(ISINSCOPE('Calendar'[Day]), BLANK(), [YoY %])

-- YTD / MTD
Продажи YTD = TOTALYTD([Продажи], 'Calendar'[Date])
Продажи MTD = TOTALMTD([Продажи], 'Calendar'[Date])

-- Прошлый месяц
Продажи прошлый месяц = CALCULATE([Продажи], PREVIOUSMONTH('Calendar'[Date]))

-- Кастомный период
Продажи 2024 = CALCULATE([Продажи], DATESBETWEEN('Calendar'[Date], DATE(2024,1,1), DATE(2024,12,31)))

-- Динамический заголовок
Заголовок = "Продажи: " & FORMAT([Продажи], "#,##0 ₽") & "  |  YoY: " & FORMAT([YoY %], "+0%;-0%;0%")
```

> **Функции дат:**
> - `SAMEPERIODLASTYEAR` — прошлый год
> - `PREVIOUSMONTH` — прошлый месяц
> - `TOTALYTD/MTD/QTD` — с начала периода
> - `DATESBETWEEN` — произвольный диапазон
> - `ISINSCOPE` — контроль отображения по уровням иерархии

## 8. Агрегация и ранжирование

```dax
-- По связанным таблицам
TotalByCategory = SUMX(RELATEDTABLE(Category), Category[Value])

-- Топ-N
Продажи Топ-5 = CALCULATE([Продажи], TOPN(5, ALL(Products), [Продажи]))

-- Без нулей
Продажи без нулей = CALCULATE([Продажи], Sales[Amount] > 0)

-- Ранжирование
Rank = RANKX(ALL(Products), [Продажи], , DESC)

-- Процентиль
90-й процентиль = PERCENTILEX.INC(Sales, [Продажи], 0.9)

-- Среднее по выражению
Средняя цена = AVERAGEX(Sales, DIVIDE([Продажи], [Количество]))
```

## 9. Мера vs Вычисляемый столбец

| Мера                     | Вычисляемый столбец                  |
|--------------------------|--------------------------------------|
| Динамична (реагирует на фильтры) | Статична (считается при загрузке)    |
| Не занимает память       | Занимает место в модели              |
| Используй: YoY, % от общего, фильтры | Используй: категории, флаги, ID      |

> **Правило:** Если зависит от **контекста** — **мера**. Если **фиксированное значение** — **столбец**.

## 10. Производительность

```dax
-- МЕДЛЕННО:
SUMX(Sales, Sales[Quantity] * Sales[Price])

-- БЫСТРО:
SUM(Sales[TotalAmount]) -- посчитай в Power Query
```

> Избегай `SUMX` на больших таблицах

## 11. Шаблон сложной меры

```dax
Продажи по категориям =
VAR CurrentCategory = SELECTEDVALUE(Products[Category])
VAR TotalSales = [Продажи]
VAR CategorySales = CALCULATE([Продажи], ALL(Products), Products[Category] = CurrentCategory)
VAR Percentage = DIVIDE(CategorySales, TotalSales)
RETURN
SWITCH(
    TRUE(),
    ISBLANK(CurrentCategory), TotalSales,
    Percentage < 0.1, "Меньше 10%",
    "Основная категория"
)
```

---
**Источники:** DAX Guide, Power BI Community.  
*Добавляйте свои меры через PR.*